2) The continuous heterogeneity, random coefficients model. If you are having difficulties coding, please do the following:
 a) simplify the covariance matrix by only estimating the diagonal elements of the Cholesky matrix
 b) start with a subset of the parameters and allow them to vary across households instead of all 5.

3) Do the above with different sets of draws 10, 200, 300, 400 and 500. Instead of inspecting the parameters, first compute the elasticities and see how much they change with increasing numbers of draws.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt

# Data Loading and Analysis

In [2]:
yogurt_df = pd.read_csv("../datasets/Yogurt.csv")

In [3]:
yogurt_df.describe()

,Pan I.D.,Expend $,Income,HH Size,IPT,Quantity,Brand 1,Brand 2,Brand 3,Brand 4,Feature 1,Feature 2,Feature 3,Feature 4,Price 1,Price 2,Price 3,Price 4,PanelistFirstObs
count,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000
mean,48.607819,36.476062,8.720988,2.802058,4.533745,2.585597,0.341975,0.401235,0.029218,0.227572,0.055556,0.039506,0.037449,0.037449,0.106248,0.081532,0.053622,0.079507,0.041152
std,27.858116,31.673291,3.800654,1.173291,14.930892,1.869089,0.474469,0.490249,0.168452,0.419351,0.229109,0.194836,0.189897,0.189897,0.020587,0.011047,0.008054,0.007714,0.198683
min,1.000000,-0.100000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.012000,0.000000,0.025000,0.004000,0.000000
25%,23.000000,12.092500,6.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.103000,0.081000,0.050000,0.079000,0.000000
50%,47.000000,27.485001,9.000000,3.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.108000,0.086000,0.054000,0.079000,0.000000
75%,73.000000,51.712500,12.000000,4.000000,5.000000,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.115000,0.086000,0.061000,0.086000,0.000000
max,100.000000,286.720000,14.000000,6.000000,358.000000,14.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.193000,0.111000,0.086000,0.104000,1.000000


In [4]:
yogurt_df.head()

,Pan I.D.,Expend $,Income,HH Size,IPT,Quantity,Brand 1,Brand 2,Brand 3,Brand 4,Feature 1,Feature 2,Feature 3,Feature 4,Price 1,Price 2,Price 3,Price 4,PanelistFirstObs
0,1,40.900002,9,2,5,2,0,0,0,1,0,0,0,0,0.108,0.081,0.061,0.079,1
1,1,16.809999,9,2,5,2,0,1,0,0,0,0,0,0,0.108,0.098,0.064,0.075,0
2,1,4.060000,9,2,1,2,0,1,0,0,0,0,0,0,0.108,0.098,0.061,0.086,0
3,1,34.459999,9,2,4,2,0,1,0,0,0,0,0,0,0.108,0.098,0.061,0.086,0
4,1,8.390000,9,2,7,2,0,1,0,0,0,0,0,0,0.125,0.098,0.049,0.079,0


In [48]:
df = pd.read_csv("../datasets/Yogurt100N.csv")
Xtab=df.groupby(["Pan I.D."]).mean()


# Random coefficients model

In [30]:
def optimization_func(x, D):
    a1, a2, a3, bf, bp, a, b, c, d, e, f, g, h, i, j, k, l, m, n, p = x
    
    utm = np.array([
        [a, b, c, d, e],
        [0, f, g, h, i],
        [0, 0, j, k, l],
        [0, 0, 0, m, n],
        [0, 0, 0, 0, p]]
    )
    
    np.random.seed = 100
    Q = np.random.normal(0, 1, D*5).reshape(D, 5)
    
    global Q_gt, lik_
    Q_gt = Q@utm + np.array([a1, a2, a3, bf, bp])
    
    lik_ = pd.DataFrame({"ID": yogurt_df["Pan I.D."]})
    b1_ = pd.DataFrame({"ID": yogurt_df["Pan I.D."]})
    b2_ = pd.DataFrame({"ID": yogurt_df["Pan I.D."]})
    b3_ = pd.DataFrame({"ID": yogurt_df["Pan I.D."]})
    b4_ = pd.DataFrame({"ID": yogurt_df["Pan I.D."]})
                       
    a1_ = pd.DataFrame({"ID": yogurt_df["Pan I.D."]})
    a2_ = pd.DataFrame({"ID": yogurt_df["Pan I.D."]})
    a3_ = pd.DataFrame({"ID": yogurt_df["Pan I.D."]})
    bf_ = pd.DataFrame({"ID": yogurt_df["Pan I.D."]})
    bp_ = pd.DataFrame({"ID": yogurt_df["Pan I.D."]})
                       
    for i in range(D):
        coef = Q_gt[i]
        v1 = np.exp(coef[0] + coef[3] * yogurt_df['Feature 1'] + coef[4] * yogurt_df['Price 1'])
        v2 = np.exp(coef[1] + coef[3] * yogurt_df['Feature 2'] + coef[4] * yogurt_df['Price 2'])
        v3 = np.exp(coef[2] + coef[3] * yogurt_df['Feature 3'] + coef[4] * yogurt_df['Price 3'])
        v4 = np.exp(0 + coef[3] * yogurt_df['Feature 4'] + coef[4] * yogurt_df['Price 4'])
        denom = v1 + v2 + v3 + v4

        lik_[i] = (v1*yogurt_df['Brand 1'] + v2*yogurt_df['Brand  2'] + v3*yogurt_df['Brand 3'] + v4*yogurt_df['Brand 4']) / denom
        b1_[i] = v1 / denom
        b2_[i] = v2 / denom
        b3_[i] = v3 / denom
        b4_[i] = v4 / denom
        a1_[i] = coef[0]
        a2_[i] = coef[1]
        a3_[i] = coef[2]
        bf_[i] = coef[3]
        bp_[i] = coef[4]
    
    global lik_agg, b1_agg, b2_agg, b3_agg, b4_agg, a1_agg, a2_agg, a3_agg, bf_agg, bp_agg 
    lik_agg = lik_.groupby("ID").prod()
    b1_agg = b1_.groupby("ID").mean()
    b2_agg = b2_.groupby("ID").mean()
    b3_agg = b3_.groupby("ID").mean()
    b4_agg = b4_.groupby("ID").mean()
    a1_agg = a1_.groupby("ID").mean()
    a2_agg = a2_.groupby("ID").mean()
    a3_agg = a3_.groupby("ID").mean()
    bf_agg = bf_.groupby("ID").mean()
    bp_agg = bp_.groupby("ID").mean()
                   
    a1_agg = a1_agg * lik_agg / (lik_agg.sum(axis=0))
    a2_agg = a2_agg * lik_agg / (lik_agg.sum(axis=0))
    a3_agg = a3_agg * lik_agg / (lik_agg.sum(axis=0))
    bf_agg = bf_agg * lik_agg / (lik_agg.sum(axis=0))
    bp_agg = bp_agg * lik_agg / (lik_agg.sum(axis=0))
                   
    lik_agg["avg"] = lik_agg.mean(axis=1)
    b1_agg = b1_agg.mean(axis=1)
    b2_agg = b2_agg.mean(axis=1)
    b3_agg = b3_agg.mean(axis=1)
    b4_agg = b4_agg.mean(axis=1)
    a1_agg = a1_agg.mean(axis=1)
    a2_agg = a2_agg.mean(axis=1)
    a3_agg = a3_agg.mean(axis=1)
    bf_agg = bf_agg.mean(axis=1)
    bp_agg = bp_agg.mean(axis=1)
                   
    log_loss = np.log(lik_agg["avg"])
    total_ll = np.sum(log_loss)

    return -total_ll


In [49]:
a1, a2, a3, bf, bp, a, b, c, d, e, f, g, h, i, j, k, l, m, n, p = 1.388, 0.644, -3.086, 0.487, -37.058, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
x_init = np.array([a1, a2, a3, bf, bp, a, b, c, d, e, f, g, h, i, j, k, l, m, n, p])
output = opt.minimize(optimization_func, x_init, (10), tol=1e-20, method='L-BFGS-B')

In [32]:
output

      fun: 1805.4835567657562
 hess_inv: <20x20 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 2.44893783e+10, -7.80925811e+09,  1.30832208e+10,  2.79218880e+09,
        6.50557125e+09,  1.69414662e+10, -2.05643345e+09,  8.83864339e+08,
        2.19466505e+10,  3.39127727e+10,  1.12859092e+10,  2.40929416e+10,
        2.75757691e+09, -3.93493535e+09,  1.38335835e+08, -2.60406810e+09,
        1.64951131e+10, -8.58137569e+09,  3.40499936e+10,  1.45987438e+10])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 525
      nit: 3
     njev: 25
   status: 0
  success: True
        x: array([  1.38800011,   0.64400007,  -3.08600019,   0.48699986,
       -37.05799998,   0.99999998,   0.99999997,   0.99999996,
         0.99999992,   0.99999997,   1.        ,   0.99999993,
         0.99999995,   1.00000008,   1.00000009,   0.99999985,
         0.99999994,   0.99999995,   0.99999991,   1.00000007])